## Problem 2: A charge with a burst of sudden acceleration

A charge is initially at rest at the origin. At t=0 the charge starts to move along the x direction with constant acceleration. The acceleration lasts for 1 ns and the charge reaches a speed of 0.5c. Afterwards the charge moves with the constant speed of 0.5c. Plot out the electric field of the charge on the xz plane at different times and make an animation. Is your finding consistent with your expectation?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pycharge as pc
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors

c = 3e8  # Speed of light in meters per second
acceleration_duration = 1e-9  # Acceleration time in seconds
final_velocity = 0.5 * c  # Final velocity after acceleration
acceleration = final_velocity / acceleration_duration  # Constant acceleration

# Define the simulation grid
lim = 5.0
grid_size = 1000
x0 = np.linspace(-lim, lim, grid_size)
x, y, z = np.meshgrid(x0, 0, x0, indexing='ij')

# Time settings for the animation
tmax = 20 * acceleration_duration  # Total time to show in the animation
n_frame = 100
dt = tmax / n_frame

# Charge initialization with LinearAcceleratingCharge
charge = pc.LinearAcceleratingCharge(acceleration=acceleration, stop_t=acceleration_duration)

# Create simulation object
simulation = pc.Simulation(charge)

In [ ]:
# Directory for plots
plotdir = './plot'
if not os.path.exists(plotdir):
    os.makedirs(plotdir)

for i in range(n_frame + 1):
    t = np.array([i * dt])  # Ensure t is always an array
    E_total = simulation.calculate_E(t=t, x=x, y=y, z=z, pcharge_field='Total')
    Ex = E_total[0][:, 0, :]
    Ez = E_total[2][:, 0, :]
    Enorm = np.sqrt(Ex**2 + Ez**2)
    
    fig, ax = plt.subplots(figsize=(7, 6))
    vmin, vmax = 1e-11, 1e-8
    im = ax.pcolormesh(x0, x0, Enorm.T, norm=matplotlib.colors.LogNorm(vmin=vmin, vmax=vmax))
    stm = plt.streamplot(x0, x0, Ex.T, Ez.T, density=2, color='k')
    pos = ax.scatter(charge.xpos(t), 0, s=20, c='red', marker='o')  # Now t is an array
    ax.set_aspect('equal')
    plt.xlabel("x")
    plt.ylabel("z")
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="4%", pad=0.05)
    plt.colorbar(im, cax=cax)
    plt.title(f'Electric Field at t = {t[0]:.3e} s')
    plt.savefig(os.path.join(plotdir, f'slice{str(i).zfill(3)}.png'))
    plt.close()

In [ ]:
# Generate animation using ffmpeg and clean up the directory
os.system("ffmpeg -y -f image2 -r 12 -i plot/slice%03d.png -c:v libx264 -crf 18 -pix_fmt yuv420p acceleration_v_0.5c.mp4")
os.system("rm -rf plot")